In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import datetime as dt
import json
import requests
import schedule
import time
import tweepy
def run_algo():
    url = 'https://api.binance.com/api/v3/klines'
    symbol = 'BTCUSDT'
    interval = '1h'
    
    start = str(int(dt.datetime(2023,1,15).timestamp()*1000))
    end = str(int(dt.datetime(2023,2,15).timestamp()*1000))
    par = {'symbol': symbol, 'interval': interval, 'startTime': start, 'endTime': end, 'limit': 1000}
    btc = pd.DataFrame(json.loads(requests.get(url, params= par).text))
    btc.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
    btc.drop(columns=['open', 'high', 'low', 'volume', 'close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore'], inplace = True) #we will only use close price
    btc.index = [dt.datetime.fromtimestamp(x/1000.0) for x in btc.datetime]
    btc=btc.astype(float)

    #algo
    btc['10d-ma'] = btc['close'].rolling(10).mean().shift()
    btc['20d-ma'] = btc['close'].rolling(20).mean().shift()

    #entries
    btc['signal'] = np.where(btc['10d-ma'] > btc['20d-ma'], 1, 0) #long signal
    btc['signal'] = np.where(btc['10d-ma'] < btc['20d-ma'], -1, btc['signal']) #short signal
    btc.dropna(inplace = True)
    btc['entry'] =  btc.signal.diff()

    plt.rcParams['figure.figsize'] = 19, 9
    plt.grid(True, alpha = .3)
    plt.plot(btc.iloc[-180:]['close'], label = 'btc')
    plt.plot(btc.iloc[-180:]['10d-ma'], label = '10d-ma')
    plt.plot(btc.iloc[-180:]['20d-ma'], label = '20d-ma')
    plt.plot(btc[-180:].loc[btc.entry == 2].index, btc[-180:]['close'][btc.entry == 2], '^',
             color = 'g', markersize = 8)
    plt.plot(btc[-180:].loc[btc.entry == -2].index, btc[-180:]['close'][btc.entry == -2], 'v',
             color = 'r', markersize = 8)
    plt.legend(loc=2);
    plt.savefig("btc_ma_1h.jpg")
    plt.clf()
    
    ############################################## bybit
    api_key = 'apikey'
    secret_key = 'secretkey'
    
    endpoint = 'https://testnet.bybit.com'
    
    #signature for the api request
    def generate_signature(api_key, secret_key, verb, path, nonce, data):
        message = verb + path + str(nonce) + data
        signature = hmac.new(bytes(secret_key, 'latin-1'), msg=bytes(message, 'latin-1'), digestmod=hashlib.sha256).hexdigest()
        return signature
    
    #order place function
    def place_order(api_key, secret_key, symbol, side, order_type, qty, price, leverage):
        path = '/api/v2/order'
        verb = 'POST'
        nonce = int(time.time() * 1000)
        data = json.dumps({
            'symbol': symbol,
            'side': side,
            'order_type': order_type,
            'qty': qty,
            'price': price,
            'leverage': leverage
        })
        headers = {
            'Content-Type': 'application/json',
            'X-MBX-APIKEY': api_key,
        }
        signature = generate_signature(api_key, secret_key, verb, path, nonce, data)
        headers['X-MBX-SIGNATURE'] = signature
        url = endpoint + path
        response = requests.post(url, headers=headers, json=json.loads(data))
        return response.json()
    
    # order close function
    def close_order(api_key, secret_key, order_id):
        path = '/api/v2/order/cancel'
        verb = 'POST'
        nonce = int(time.time() * 1000)
        data = json.dumps({
            'order_id': order_id,
        })
        headers = {
            'Content-Type': 'application/json',
            'X-MBX-APIKEY': api_key,
        }
        signature = generate_signature(api_key, secret_key, verb, path, nonce, data)
        headers['X-MBX-SIGNATURE'] = signature
        url = endpoint + path
        response = requests.post(url, headers=headers, json=json.loads(data))
        return response.json()    

    ############################################## posting on twitter and open position on bybit if signal changes
    in_a_position = 0
    
    if(btc['signal'][-1] != btc['signal'][-2]):
        if(in_a_position == 1):
            close_order(api_key, secret_key, order_id)
        if(btc['signal'][-1] == 1):
            #twitter post title
            pos = 'btc/usdt long  @ '
            price = str(btc['close'][-1])
            algo = '\n(moving avg 1h)'
            text = pos+price+algo
            
            #opening the position on bybit (long)
            response = place_order(api_key, secret_key, 'BTCUSD', 'Buy', 'Market', 0.1, None, 10)
            print(response)
            order_id = response['result']['order_id']
            
        else:
            #twitter post title
            pos='btc/usdt short  @ '
            price=str(btc['close'][-1])
            entry=pos+price
            algo = '\n(moving avg 1h)'
            text=pos+price+algo
            
            #opening the position on bybit (short)
            response = place_order(api_key, secret_key, 'BTCUSD', 'Sell', 'Market', 0.1, None, 10)
            print(response)
            order_id = response['result']['order_id']
            
            
        #posting on twitter
        # Enter your API credentials
        consumer_key = 'CIgKe3bpazc1lfaF2xuzZnohg'
        consumer_secret = 'gtdcNVobl3z1WE6jUPyeezD4KgDgV3QftbCf56LL4ZuxuHxwkW'
        access_token = '1610131483963920389-krQcBDqi4oC3ApeJrvUIBIDzmwBUHp'
        access_token_secret = 'sLHSWtLCdlw7m4YyH7h3Wd2IghQemKPiKL2xkBJ4Vof5y'

        # Authenticate with the Twitter API using your API credentials
        auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)

        # Connect to the Twitter API using the authenticate handler
        api = tweepy.API(auth)

        # Open the image file
        image_path = 'btc_ma_1h.jpg'
        image = open(image_path, 'rb')

        # Set the status update text
        status_text = text

        # Upload the image to Twitter
        upload_response = api.media_upload(image_path)
        media_id = upload_response.media_id_string

        # Publish the tweet with the image and status update
        status = api.update_status(status=status_text, media_ids=[media_id]) 
    
schedule.every().hour.at(":00").do(run_algo)
while True:
    schedule.run_pending()
    time.sleep(0.99)

KeyboardInterrupt: 